<a href="https://colab.research.google.com/github/Anand-1932/Crop_Prediction/blob/main/Crop_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Problem Statement**
Here we have dataset which contain information about the soil like nitrogen level, phasphorous level, potassium level, and tempreture, rainfall in that place and we have to predict which crop will cultivated according to given information.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv("/content/Crop_recommendation.csv")

In [ ]:
df



# *   **N :-** Nitrogen level of the soil

# *   **p :-** Phosphoros level of the soil

# *   **k :-** Potassium level of the soil

# *   **Tempreture :-** Tempreture of the area
# *   **Humidity :-** Humidity of the area


# *   **PH :-** PH value of the soil


# *   **Rainfall :-** Rainfall in that area






In [ ]:
df.shape

# ***Basic checks***

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.describe(include='all')

# ***EDA***

In [ ]:
plt.figure(figsize = (14,4))
plt.subplot(121)
df['N'].value_counts().plot(kind='kde')
plt.title('Nitrogen Level')

plt.subplot(122)
sns.distplot(df['N'])
plt.title('Nitrogen Level')

* Data is right skewed
* Most of value range between 0 to 50
* Min value is 0 and Maximum value is near 140





In [ ]:
plt.figure(figsize = (14,4))
plt.subplot(121)
df['P'].value_counts().plot(kind='kde')
plt.title('Phosphrous Level')

plt.subplot(122)
sns.distplot(df['P'])
plt.title('Phosphrous Level')

* Data is not distributed normally
* Most of the value range between 0 to 100.
* Minimum value is near 0 and maximum value is near 150

In [ ]:
plt.figure(figsize=(14,4))

plt.subplot(121)
df['K'].value_counts().plot(kind = 'kde')
plt.title('Potassium Level')

plt.subplot(122)
sns.distplot(df['K'])
plt.title('Potassium Level')

* Data is right skewed
* Most of the value range between 0 to 50
* Minimum value is near 0 and maximum value is near 205

In [ ]:
plt.figure(figsize=(14,4))

plt.subplot(122)
sns.distplot(df['temperature'])
plt.title("Tempreture")

* data is not completly normal
* Most of the data point lie between 15 and 35
* Minimun value is near 0 and maximum value is near 50

In [ ]:
plt.figure(figsize=(14,4))

plt.subplot(122)
sns.distplot(df['humidity'])
plt.title("Humidity")

* Data is left skewed.
* Most of the data point lie between 60 and 100.
* Minimum value is near 15 and maximum value is near 100.

In [ ]:
plt.figure(figsize=(14,4))

plt.subplot(122)
sns.distplot(df['ph'])
plt.title("ph")

* Data point is not normally distributed.
* Most of the value lie between 5.5 and 8.
* Minimum value is near 3 and Maximum value is near 10.

In [ ]:
plt.figure(figsize = (14,4))


plt.subplot(122)
sns.distplot(df['rainfall'])
plt.title('Rainfall (cm)')

* data point is right skewed.
* Rainfall in the area mostly lie between 40 and 150
* Minimum rainfall in the area is near 40 and maximum value in the area is near 300  

In [ ]:
sns.barplot(x=df['N'],y=df['label'])

In [ ]:
sns.barplot(x=df['P'],y=df['label'])

In [ ]:
sns.barplot(x=df['K'],y=df['label'])

In [ ]:
sns.barplot(x=df['temperature'],y=df['label'])

In [ ]:
sns.barplot(x=df['humidity'],y=df['label'])

In [ ]:
sns.barplot(x=df['ph'],y=df['label'])

In [ ]:
sns.barplot(x=df['rainfall'],y=df['label'])

In [ ]:
sns.scatterplot(x=df['N'],y=df['K'],hue=df['label'])

In [ ]:
sns.scatterplot(x=df['N'],y=df['P'],hue=df['label'])

# ***Data Preprocessing***

In [ ]:
df.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
plotnumber = 1
for col in df.columns:
  if plotnumber<=7:
    ax = plt.subplot(4,2,plotnumber)
    sns.boxplot(x = df[col])
    plt.xlabel(col,fontsize=10)
    plt.ylabel('count',fontsize=10)

    plotnumber+=1

  plt.tight_layout()

In [ ]:
# creating a wisker function which will reutrn the lower and upper wisker

def wisker(col):
  Q1,Q3 = np.percentile(col,[25,75])
  IQR = Q3 - Q1
  lw = Q1-1.5*IQR
  uw = Q3+1.5*IQR
  return lw,uw

In [ ]:
# Applying wisker function on columns which contain outliers

for i in ['P','K','temperature','humidity','ph','rainfall']:
  lw,uw = wisker(df[i])
  df[i] = np.where(df[i]<lw,lw,df[i])
  df[i] = np.where(df[i]>uw,uw,df[i])


In [ ]:
plt.figure(figsize=(14,10))
plotnumber = 1
for col in df.columns:
  if plotnumber<=7:
    ax = plt.subplot(4,2,plotnumber)
    sns.boxplot(x = df[col])
    plt.xlabel(col,fontsize=10)
    plt.ylabel('count',fontsize=10)

    plotnumber+=1

  plt.tight_layout()

# Splitting data into test and train dataset

In [ ]:
x= df.drop('label',axis=1)
y = df['label']

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.25,random_state=5)

In [ ]:
x_train

In [ ]:
rf =RandomForestClassifier(n_estimators=100,max_depth=35, min_samples_split=2, min_samples_leaf=1)
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
print('Accuracy',accuracy_score(y_test,y_pred))
print('Classification_Report',classification_report(y_test,y_pred))

In [ ]:
import pickle
pickle.dump(rf,open('model.pkl','wb'))

In [ ]:
# Now we will apply standard scaler and create pipeline
# step1 = ColumnTransformer(transformers=[
#     ('col_trf',StandardScaler(),[0,1,2,3,4,5,6])
# ],remainder = 'passthrough')

# step2 = LogisticRegression()

# pipe = Pipeline([
#     ('step1',step1),
#     ('step2',step2)
# ])

# pipe.fit(x_train,y_train)

# y_pred = pipe.predict(x_test)

# print('Accuracy',accuracy_score(y_test,y_pred))
# print('Classification_Report',classification_report(y_test,y_pred))



In [ ]:
# Now we will apply standard scaler and create pipeline
# step1 = ColumnTransformer(transformers=[
#     ('col_trf',StandardScaler(),[0,1,2,3,4,5,6])
# ],remainder = 'passthrough')

# step2 = RandomForestClassifier(n_estimators=100,max_depth=35, min_samples_split=2, min_samples_leaf=1)

# pipe = Pipeline([
#     ('step1',step1),
#     ('step2',step2)
# ])

# pipe.fit(x_train,y_train)

# y_pred = pipe.predict(x_test)

# print('Accuracy',accuracy_score(y_test,y_pred))
# print('Classification_Report',classification_report(y_test,y_pred))


In [ ]:
# Now we will apply standard scaler and create pipeline
# step1 = ColumnTransformer(transformers=[
#     ('col_trf',StandardScaler(),[0,1,2,3,4,5,6])
# ],remainder = 'passthrough')

# step2 = DecisionTreeClassifier(max_depth=35, splitter='best',min_samples_split=2, min_samples_leaf=1)

# pipe = Pipeline([
#     ('step1',step1),
#     ('step2',step2)
# ])

# pipe.fit(x_train,y_train)

# y_pred = pipe.predict(x_test)

# print('Accuracy',accuracy_score(y_test,y_pred))
# print('Classification_Report',classification_report(y_test,y_pred))

In [ ]:
# import pickle

# pickle.dump(df,open('data.pkl','wb'))
# pickle.dump(pipe,open('pipe.pkl','wb'))